In [116]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from scipy import sparse

In [44]:
genesMappingGO = pd.read_csv('../processed_data/genesMappingGO.csv')
genesMappingGO.head()

,gene,goId
0,AAC1,"['GO:0005471', 'GO:0006783', 'GO:0006839', 'GO..."
1,AAC3,"['GO:0005471', 'GO:0009061', 'GO:0015886', 'GO..."
2,AAD10,"['GO:0047681', 'GO:0006081', 'GO:0005575', 'GO..."
3,AAD14,"['GO:0047681', 'GO:0006081', 'GO:0005575', 'GO..."
4,AAD15,"['GO:0047681', 'GO:0006081', 'GO:0005575', 'GO..."


In [45]:
genesMappingGO.goId = genesMappingGO.goId.apply(lambda x: eval(x))

In [46]:
genesMappingGO.head()

,gene,goId
0,AAC1,"[GO:0005471, GO:0006783, GO:0006839, GO:000906..."
1,AAC3,"[GO:0005471, GO:0009061, GO:0015886, GO:005508..."
2,AAD10,"[GO:0047681, GO:0006081, GO:0005575, GO:004783..."
3,AAD14,"[GO:0047681, GO:0006081, GO:0005575, GO:004783..."
4,AAD15,"[GO:0047681, GO:0006081, GO:0005575, GO:004783..."


In [47]:
goDescription = pd.read_csv('../processed_data/goDescription.csv')
goDescription.head()

,goId,family
0,GO:0003674,F
1,GO:0017116,F
2,GO:0120133,P
3,GO:0003724,F
4,GO:0015035,F


In [48]:
goDescription.goId.duplicated().sum()

0

In [49]:
exp = genesMappingGO.explode('goId')
exp.head()

,gene,goId
0,AAC1,GO:0005471
0,AAC1,GO:0006783
0,AAC1,GO:0006839
0,AAC1,GO:0009060
0,AAC1,GO:0015886


In [50]:
exp.shape

(52657, 2)

In [53]:
exp.duplicated().sum()

0

In [66]:
familyGeneGoId = exp.merge(goDescription[['goId','family']], on='goId')
familyGeneGoId.head()

,gene,goId,family
0,AAC1,GO:0005471,F
1,AAC3,GO:0005471,F
2,ODC1,GO:0005471,F
3,ODC2,GO:0005471,F
4,SAL1,GO:0005471,F


In [125]:
pivoted = {f: pd.pivot(familyGeneGoId[familyGeneGoId['family']==f][['gene','goId']] , index='gene', columns='goId', values='goId') for f in familyGeneGoId.family.unique()}

In [129]:

def convertToBool(df):
    df = df.fillna(0)
    df[df != 0] = 1
    df = df.astype(bool)
    return df
pivoted = {f: convertToBool(p) for f,p in pivoted.items()}

In [132]:
dtype = pd.SparseDtype(bool, fill_value=False)

In [133]:
for f,p in pivoted.items():
    pivoted[f] = pivoted[f].astype(dtype)

In [138]:
tfidfPivoted = {f: TfidfTransformer().fit_transform(p) for f,p in pivoted.items()}

In [139]:
import numpy as np
np.unique(np.array(tfidfPivoted['F'].todense()))

array([0.        , 0.07927117, 0.08353102, ..., 0.9567486 , 0.96046586,
       1.        ])

In [140]:
i = pivoted['F'].sum(axis=1).argmax()
s = pivoted['F'].iloc[i]

s[s].index.values.tolist()

['GO:0003824',
 'GO:0004312',
 'GO:0004313',
 'GO:0004314',
 'GO:0004317',
 'GO:0004318',
 'GO:0004319',
 'GO:0004320',
 'GO:0004321',
 'GO:0008659',
 'GO:0008693',
 'GO:0016295',
 'GO:0016296',
 'GO:0016297',
 'GO:0016409',
 'GO:0016418',
 'GO:0016419',
 'GO:0016491',
 'GO:0016631',
 'GO:0016740',
 'GO:0016787',
 'GO:0016829',
 'GO:0019171',
 'GO:0047451']

In [141]:
s.name

'FAS1'

In [142]:
sIndex = np.where(pivoted['F'].index==s.name)[0]

In [143]:
tfidfPivoted['F'][sIndex].data

array([0.22812804, 0.21746669, 0.1311322 , 0.08353102, 0.07927117,
       0.22812804, 0.10538328, 0.21746669, 0.22812804, 0.18580929,
       0.22812804, 0.22812804, 0.22812804, 0.22812804, 0.22812804,
       0.21746669, 0.22812804, 0.22812804, 0.22812804, 0.22812804,
       0.21746669, 0.22812804, 0.21746669, 0.1044138 ])

In [144]:
import pickle as pkl
with open('../processed_data/geneOntologyDataset.pkl', 'wb') as out:
    pkl.dump([pivoted, tfidfPivoted], out)
    

In [145]:
import pickle as pkl
with open('../processed_data/geneOntologyDataset.pkl', 'rb') as inp:
    [pivoted, tfidfPivoted] = pkl.load(inp)
    